In [1]:
import pandas as pd
from nltk.stem.porter import *
import ast
from heapq import nlargest
from operator import itemgetter

df = pd.read_json("image_name_and_cuisine_name.json")
dfIng = pd.read_csv('ingrefromIm.csv')
kaggle_dataset = pd.read_csv("kaggle_dataset_one_word.csv") # new2.csv, kaggle_dataset_one_word.csv

In [2]:
df

,id,title,cuisine
0,0eea6409f9,Italian Orange Cookies,italian
1,d837c7bceb,Italian Orange Cookies,italian
2,e1c1ecfb09,Italian Orange Cookies,italian
3,5fb091a1b5,Pate a Choux Shells (French Pastry Shells) Recipe,french
4,4aa71d889a,Italian Vegetarian Patties,italian
...,...,...,...
6290,16df7f35f4,Italian Meatballs,italian
6291,b464ebf06a,Irish Coffee,irish
6292,5078429971,Korean-Style Lettuce and Onion Salad,korean
6293,185af6be99,Thai Fling,thai


In [3]:
dfIng = dfIng.rename(columns={"Name": "id"}, errors="raise")
dfIng

,Unnamed: 0,id,Ingredients
0,0,6a15d0b4d7,"['onion', 'water', 'salt', 'carrot', 'chicken'..."
1,1,9edf83d5a4,"['banana', 'kiwi', 'strawberries', 'yogurt', '..."
2,2,1aa03fa7c6,"['oil', 'quinoa', 'pepper', 'avocado', 'salt',..."
3,3,272fe84aa6,"['pepper', 'chicken', 'salt', 'onion', 'garlic..."
4,4,c4525a9989,"['sugar', 'flour', 'salt', 'butter', 'water', ..."
...,...,...,...
6261,6261,33cae617e1,"['sugar', 'egg', 'cheese', 'butter', 'cream', ..."
6262,6262,e53162f978,"['salt', 'flour', 'baking_powder', 'baking_sod..."
6263,6263,7e949a0d75,"['cream', 'strawberries', 'sugar', 'pineapple'..."
6264,6264,adc64c4c73,"['cheese', 'lettuce', 'beans', 'salsa', 'onion..."


In [4]:
mergedDf = pd.merge(df, dfIng, on='id')
mergedDf

,id,title,cuisine,Unnamed: 0,Ingredients
0,0eea6409f9,Italian Orange Cookies,italian,1392,"['sugar', 'flour', 'egg', 'butter', 'salt', 'b..."
1,d837c7bceb,Italian Orange Cookies,italian,357,"['sugar', 'egg', 'flour', 'butter', 'salt', 'm..."
2,e1c1ecfb09,Italian Orange Cookies,italian,279,"['sugar', 'flour', 'butter', 'salt', 'baking_p..."
3,5fb091a1b5,Pate a Choux Shells (French Pastry Shells) Recipe,french,4646,"['pepper', 'chicken', 'mayonnaise', 'roll', 'o..."
4,4aa71d889a,Italian Vegetarian Patties,italian,197,"['cheese', 'egg', 'pepper', 'oil', 'breadcrumb..."
...,...,...,...,...,...
6290,16df7f35f4,Italian Meatballs,italian,6102,"['cheese', 'onion', 'pepper', 'beef', 'clove',..."
6291,b464ebf06a,Irish Coffee,irish,3708,"['water', 'sugar', 'coffee', 'orange']"
6292,5078429971,Korean-Style Lettuce and Onion Salad,korean,26,"['oil', 'onion', 'vinegar', 'lettuce', 'pepper..."
6293,185af6be99,Thai Fling,thai,2522,"['juice', 'ice', 'lime', 'rum', 'tequila']"


In [5]:
merged_Recipe1M = mergedDf.drop_duplicates(subset='id', keep="last")
merged_Recipe1M

,id,title,cuisine,Unnamed: 0,Ingredients
0,0eea6409f9,Italian Orange Cookies,italian,1392,"['sugar', 'flour', 'egg', 'butter', 'salt', 'b..."
1,d837c7bceb,Italian Orange Cookies,italian,357,"['sugar', 'egg', 'flour', 'butter', 'salt', 'm..."
2,e1c1ecfb09,Italian Orange Cookies,italian,279,"['sugar', 'flour', 'butter', 'salt', 'baking_p..."
3,5fb091a1b5,Pate a Choux Shells (French Pastry Shells) Recipe,french,4646,"['pepper', 'chicken', 'mayonnaise', 'roll', 'o..."
4,4aa71d889a,Italian Vegetarian Patties,italian,197,"['cheese', 'egg', 'pepper', 'oil', 'breadcrumb..."
...,...,...,...,...,...
6290,16df7f35f4,Italian Meatballs,italian,6102,"['cheese', 'onion', 'pepper', 'beef', 'clove',..."
6291,b464ebf06a,Irish Coffee,irish,3708,"['water', 'sugar', 'coffee', 'orange']"
6292,5078429971,Korean-Style Lettuce and Onion Salad,korean,26,"['oil', 'onion', 'vinegar', 'lettuce', 'pepper..."
6293,185af6be99,Thai Fling,thai,2522,"['juice', 'ice', 'lime', 'rum', 'tequila']"


In [6]:
#Configuration
cuisine = 'american'
top = 50

In [7]:
selected_cuisine = merged_Recipe1M.loc[merged_Recipe1M['cuisine'] == cuisine]
selected_cuisine
selected_cuisine['Ingredients']
bag_of_ing_Recipe1M = {}
for list_of_ing in selected_cuisine['Ingredients']:
    # Converting string that corresponds to list, to a list
    list_of_ing = ast.literal_eval(list_of_ing)
    list_of_ing = [n.strip() for n in list_of_ing]
    for ing in list_of_ing:
        if ing not in bag_of_ing_Recipe1M:
            bag_of_ing_Recipe1M[ing] = 0
        bag_of_ing_Recipe1M[ing] += 1

selected_cuisine = kaggle_dataset.loc[kaggle_dataset['cuisine'] == cuisine]
selected_cuisine
selected_cuisine['ingredients']
bag_of_ing_kaggleDS = {}
for list_of_ing in selected_cuisine['ingredients']:
    # Converting string that corresponds to list, to a list
    list_of_ing = ast.literal_eval(list_of_ing)
    list_of_ing = [n.strip() for n in list_of_ing]
    for ing in list_of_ing:
        if ing not in bag_of_ing_kaggleDS:
            bag_of_ing_kaggleDS[ing] = 0
        bag_of_ing_kaggleDS[ing] += 1

               
result_dataframe = {'Recipe1M_name': [], 'Recipe1M_count': [], 'Kaggle_name': [], 'Kaggle_count': []}
for name, number in nlargest(top, bag_of_ing_Recipe1M.items(), key=itemgetter(1)):
    result_dataframe['Recipe1M_name'].append(name)
    result_dataframe['Recipe1M_count'].append(number)
    
for name, number in nlargest(top, bag_of_ing_kaggleDS.items(), key=itemgetter(1)):
    result_dataframe['Kaggle_name'].append(name)
    result_dataframe['Kaggle_count'].append(number)

result_dataframe = pd.DataFrame(data=result_dataframe)

In [8]:
print('cuisine: ' + cuisine)
result_dataframe

cuisine: american


,Recipe1M_name,Recipe1M_count,Kaggle_name,Kaggle_count
0,pepper,50,salt,3885
1,salt,48,pepper,3124
2,onion,35,flour,2160
3,oil,35,sugar,2123
4,clove,26,oil,2026
5,cheese,25,garlic,1910
6,sugar,23,butter,1788
7,milk,23,eggs,1553
8,tomato,20,onions,1448
9,butter,16,milk,1326
